In [1]:
import gzip
import pandas as pd
import numpy as np
import pickle
import requests

In [2]:
f = open("token_TMDB.txt","r")
token = f.read()
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}

In [61]:
crew = pickle.load(open('crew.pkl', 'rb'))

In [115]:
cast = pickle.load(open('cast.pkl', 'rb'))

In [112]:
crew

,tconst,directors,writers
28800,tt0029284,{nm0437717},"{nm0855139, nm0565609, nm0437717, nm0818415, n..."
30853,tt0031359,{nm0225555},"{nm0358096, nm0092658, nm0712653}"
31467,tt0031976,{nm0715346},"{nm0188743, nm0178784, nm0930803}"
31670,tt0032179,{nm0212704},"{nm0001294, nm0303335, nm0212704}"
31672,tt0032181,{nm0188669},"{nm0792845, nm0428177}"
...,...,...,...
10060305,tt9907782,{nm1193346},{nm1193346}
10060594,tt9908390,{nm1415268},"{nm1597648, nm1597688}"
10061808,tt9911196,{nm0631590},"{nm0495599, nm0277932, nm3547655, nm2063122}"
10064086,tt9916270,{nm1480867},"{nm10538402, nm1480867}"


tconst      0
ordering    0
nconst      0
category    0
dtype: int64

In [65]:
crew['directors'] = crew['directors'].apply(lambda x: set(x.split(',')) if isinstance(x, str) else x)
crew['writers'] = crew['writers'].apply(lambda x: set(x.split(',')) if isinstance(x, str) else x)

In [66]:
writers = set(crew['writers'].explode())
directors = set(crew['directors'].explode())
actors = set(cast['nconst'].unique())

In [67]:
len(directors)+len(actors)+len(writers)
len(writers|directors|actors)

160556

In [68]:
crew

,tconst,directors,writers
28800,tt0029284,{nm0437717},"{nm0855139, nm0565609, nm0437717, nm0818415, n..."
30853,tt0031359,{nm0225555},"{nm0358096, nm0092658, nm0712653}"
31467,tt0031976,{nm0715346},"{nm0188743, nm0178784, nm0930803}"
31670,tt0032179,{nm0212704},"{nm0001294, nm0303335, nm0212704}"
31672,tt0032181,{nm0188669},"{nm0792845, nm0428177}"
...,...,...,...
10060305,tt9907782,{nm1193346},{nm1193346}
10060594,tt9908390,{nm1415268},"{nm1597648, nm1597688}"
10061808,tt9911196,{nm0631590},"{nm3547655, nm0277932, nm2063122, nm0495599}"
10064086,tt9916270,{nm1480867},"{nm10538402, nm1480867}"


In [69]:
people_ids = writers|directors|actors
len(people_ids)

160556

In [3]:
people = pickle.load(open('name.basics.pkl', 'rb'))

In [200]:
people

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0083922,tt0069467,tt0050976"
...,...,...,...,...,...,...
13420446,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13420447,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
13420448,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
13420449,nm9993718,Aayush Nair,NaN,NaN,cinematographer,tt8736744


In [201]:
people = people[people['nconst'].isin(people_ids)]

In [185]:
filt = list(pd.Series(list(people_ids))[~pd.Series(list(people_ids)).isin(people['nconst'])])

In [75]:
filt

['', 'nm2930110', 'nm1969711']

In [76]:
display(crew[crew['writers'].apply(lambda x: any(item in filt for item in x))])
display(crew[crew['directors'].apply(lambda x: any(item in filt for item in x))])
display(cast[cast['nconst'].isin(filt)])

,tconst,directors,writers
54117,tt0054745,"{nm0602841, nm0745541}",{}
57323,tt0057960,{nm0001596},{}
61496,tt0062144,{nm0164999},{}
62827,tt0063477,{nm0800140},{}
63825,tt0064482,{nm0207108},{}
...,...,...,...
9930419,tt9617716,"{nm5385537, nm3758612}",{}
9996974,tt9765144,{nm10933454},{}
9997425,tt9766160,{nm10246130},{}
10035436,tt9850370,"{nm10509355, nm9921790}",{}


,tconst,directors,writers
9855955,tt9448868,{},{}


,tconst,ordering,nconst


In [79]:
def filtering(lst, filt):
    filtered = {item for item in lst if item not in filt}
    return filtered if filtered else set()


In [80]:
crew['writers'].apply(lambda x: filtering(x, filt)).loc[[2585450, 386691]]

2585450                                              {}
386691     {nm2707225, nm1587401, nm1552261, nm1352003}
Name: writers, dtype: object

In [81]:
crew['writers'] = crew['writers'].apply(lambda x: filtering(x, filt))

In [82]:
crew['writers'].loc[[2585450, 386691]]

2585450                                              {}
386691     {nm2707225, nm1587401, nm1552261, nm1352003}
Name: writers, dtype: object

In [202]:
people = people[people['nconst'].isin(people_ids)]

In [101]:
directors_df = crew.explode(column='directors')[['tconst','directors']]
directors_df.rename(columns={'directors': 'nconst'},inplace=True)
directors_df['profession'] = 'director'

In [102]:
directors_df

,tconst,nconst,profession
28800,tt0029284,nm0437717,director
30853,tt0031359,nm0225555,director
31467,tt0031976,nm0715346,director
31670,tt0032179,nm0212704,director
31672,tt0032181,nm0188669,director
...,...,...,...
10060305,tt9907782,nm1193346,director
10060594,tt9908390,nm1415268,director
10061808,tt9911196,nm0631590,director
10064086,tt9916270,nm1480867,director


In [106]:
writers_df = crew.explode(column='writers')[['tconst','writers']]
writers_df.rename(columns={'writers': 'nconst'},inplace=True)
writers_df['profession'] = 'writers'

In [130]:
writers_df

tconst        False
nconst         True
profession    False
dtype: bool

In [122]:
cast_df = cast[['tconst','nconst','category']]
cast_df.rename(columns={'category': 'profession'},inplace=True)

/var/folders/_8/f5zjdbxn0vs8y_m3qv7n_8800000gn/T/ipykernel_73344/1173181888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cast_df.rename(columns={'category': 'profession'},inplace=True)


In [123]:
cast_df

,tconst,nconst,profession
325941,tt0029284,nm0002050,actress
325942,tt0029284,nm0000026,actor
325943,tt0029284,nm0000068,actor
325944,tt0029284,nm0665850,actress
325945,tt0029284,nm0794700,actress
...,...,...,...
85192784,tt9916362,nm10678596,actress
85192785,tt9916362,nm10678595,actress
85192786,tt9916362,nm10678597,actress
85192787,tt9916362,nm3084436,actor


In [125]:
relations = pd.concat([cast_df,directors_df,writers_df])

In [133]:
relations.dropna(inplace=True)

In [139]:
relations = relations[relations['nconst'].isin(people['nconst'])]

In [140]:
people

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0083922,tt0069467,tt0050976"
...,...,...,...,...,...,...
13419545,nm9992630,Karin Cherches,NaN,NaN,actress,tt5039088
13419614,nm9992750,Loghan Bazan,NaN,NaN,actress,tt8736336
13420288,nm9993535,Henry Lawfull,2006.0,NaN,actor,"tt10187208,tt5900600"
13420358,nm9993616,Ryan Mac Lennan,NaN,NaN,actor,tt4844148


In [141]:
relations

,tconst,nconst,profession
325941,tt0029284,nm0002050,actress
325942,tt0029284,nm0000026,actor
325943,tt0029284,nm0000068,actor
325944,tt0029284,nm0665850,actress
325945,tt0029284,nm0794700,actress
...,...,...,...
10061808,tt9911196,nm2063122,writers
10064086,tt9916270,nm10538402,writers
10064086,tt9916270,nm1480867,writers
10064131,tt9916362,nm1893148,writers


In [144]:
relations.reset_index(drop=True)

,tconst,nconst,profession
0,tt0029284,nm0002050,actress
1,tt0029284,nm0000026,actor
2,tt0029284,nm0000068,actor
3,tt0029284,nm0665850,actress
4,tt0029284,nm0794700,actress
...,...,...,...
415930,tt9911196,nm2063122,writers
415931,tt9916270,nm10538402,writers
415932,tt9916270,nm1480867,writers
415933,tt9916362,nm1893148,writers


In [145]:
relations.to_pickle('cast_crew.pkl')

In [ ]:
writers_df = crew.explode(column='writers')[['tconst','writers']]

In [193]:
movies = pickle.load(open('movies.pkl', 'rb'))

In [188]:
known_for = people.apply(lambda row: [(row['nconst'], title) for title in row['knownForTitles'].split(',')] if isinstance(row['knownForTitles'], str) else [], axis=1)
known_for = pd.DataFrame([item for sublist in known_for for item in sublist], columns=['nconst', 'knownForTitle'])

In [197]:
known_for = known_for[known_for['knownForTitle'].isin(movies['tconst'])]

In [196]:
known_for.to_pickle('known_for.pkl')

In [214]:
people.to_pickle('people.pkl')

In [215]:
people

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0083922,tt0069467,tt0050976"
...,...,...,...,...,...,...
13419545,nm9992630,Karin Cherches,NaN,NaN,actress,tt5039088
13419614,nm9992750,Loghan Bazan,NaN,NaN,actress,tt8736336
13420288,nm9993535,Henry Lawfull,2006.0,NaN,actor,"tt10187208,tt5900600"
13420358,nm9993616,Ryan Mac Lennan,NaN,NaN,actor,tt4844148


In [146]:
from sqlalchemy import create_engine
from getpass import getpass

In [147]:
password = getpass()

········


In [148]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost:3306/recommender'
engine = create_engine(connection_string)

In [204]:
with engine.connect() as conn:
    relations.to_sql(name='relations', con=engine, if_exists='replace', index=False)
    people.to_sql(name='people', con=engine, if_exists='replace', index=False)
    known_for.to_sql(name='knownFor', con=engine, if_exists='replace', index=False)

In [206]:
with engine.connect() as conn:
    